In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import statistics
from scipy.stats import pearsonr

import sys
import os

# Use os.getcwd() to get the current working directory if __file__ is not available
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

# Now you can import the necessary modules from the parent directory
from RADARDataCapture import FMCWRADARDataCapture
from MOCAPDataCapture import MOCAPDataCapture
from FPDataCapture import FPDataCapture

Unwrapped phase by Bin displacements

In [ ]:

# Now proceed with the code
# Path to the CSV file containing time labels
time_labels_df_path = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/MOCAP_FP_RADAR_FU_Stable_Break_FD_TIME_FRAMES_v3.csv'
time_labels_df = pd.read_csv(time_labels_df_path)

print(time_labels_df.head())

output_folder_path = "/Volumes/FourTBLaCie/Yoga_Study_RADAR_AoA_Stability"

problem_list = []

for index, row in time_labels_df.iterrows():
    print(row)
    
    capture_name = row['RADAR_capture']

    participant = capture_name[:2]
    if participant != "24" and participant != "15":
        continue

    radar_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_RADAR_Data_Renamed/{participant}/{capture_name}.h5"

    try:
        # Initialize the FMCWRADARDataCapture object
        RADAR_object = FMCWRADARDataCapture(file_path=radar_file_path)
    except Exception as e:
        problem_list.append((capture_name, row['tx'], e))
        continue

    # Load the data
    dataCubes = RADAR_object.load_and_save()
    
    print(dataCubes.shape)

    # Select frames between frame_stable and frame_end
    frame_stable = int(row['frame_stable'])
    if row['is_final_tx']:
        frame_end = int(row['frame_end'])
    else:
        frame_end = int(row['frame_break'])

    dataCubes_stable = dataCubes[:, frame_stable:frame_end, :, :]

    range_bins_of_interest = [7,8]

    # Compute the unwrapped phases
    unwrapped_phases = RADAR_object.unwrapped_microdoppler_phase_displacement(
        capture_name=capture_name,
        tx = row['tx'],
        dataCubes=dataCubes_stable,
        range_bins=range_bins_of_interest,
        plot=True  # Set to True to plot the unwrapped phases
    )

    # # Access the unwrapped phase data for further analysis
    # channel = 0
    # range_bin = 6
    # frequencies, times, unwrapped_phase_data = unwrapped_phases[channel][range_bin]
    print("done")


FFT Analysis

Unwrapped phase summary stats

In [2]:
import pandas as pd
import numpy as np

# Path to the CSV file containing time labels
time_labels_df_path = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/MOCAP_FP_RADAR_FU_Stable_Break_FD_TIME_FRAMES_v3.csv'
time_labels_df = pd.read_csv(time_labels_df_path)

print(time_labels_df.head())

output_folder_path = "/Volumes/FourTBLaCie/Yoga_Study_RADAR_AoA_Stability"

problem_list = []
statistics = {}  # To store summary stats per participant

for index, row in time_labels_df.iterrows():
    capture_name = row['RADAR_capture']

    participant = capture_name[:2]
    # if participant != "24" and participant != "15":  # Only analyzing participants 24 and 15
    #     continue

    radar_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_RADAR_Data_Renamed/{participant}/{capture_name}.h5"

    # Handle special cases to skip certain captures or transitions
    if capture_name == '22_MNTRL_RR_V1' and str(row['tx']) == "3":
        continue
    elif capture_name == '24_MNTRR_RR_V1' and str(row['tx']) == "2":
        continue
    elif capture_name == '12_MNTRR_RR_V1':
        continue

    
    try:
        # Initialize the FMCWRADARDataCapture object
        RADAR_object = FMCWRADARDataCapture(file_path=radar_file_path)
    except Exception as e:
        problem_list.append((capture_name, row['tx'], e))
        continue

    # Load the data
    dataCubes = RADAR_object.load_and_save()

    # Select frames between frame_stable and frame_end
    frame_stable = int(row['frame_stable'])
    if row['is_final_tx']:
        frame_end = int(row['frame_end'])
    else:
        frame_end = int(row['frame_break'])

    dataCubes_stable = dataCubes[:, frame_stable:frame_end, :, :]

    range_bins_of_interest = [7]

    # Compute the unwrapped phases or displacement
    displacements = RADAR_object.unwrapped_microdoppler_phase_displacement(
        capture_name=capture_name,
        tx = row['tx'],
        dataCubes=dataCubes_stable,
        range_bins=range_bins_of_interest,
        plot=False  # We don't need to plot for every iteration
    )

    # Compute summary statistics for each channel and range bin
    for channel in displacements:
        for range_bin in displacements[channel]:
            t, displacement = displacements[channel][range_bin]

            # Compute summary statistics
            displacement_mean = np.mean(displacement)
            displacement_std = np.std(displacement)
            displacement_range = np.ptp(displacement)  # Peak-to-peak (max - min)
            displacement_max = np.max(displacement)
            displacement_min = np.min(displacement)

            # Print the stats for the current channel, range bin, and participant
            print(f"Participant: {participant}, Capture: {capture_name}, Channel: {channel}, Range Bin: {range_bin}")
            print(f"Mean Displacement: {displacement_mean:.6f} m")
            print(f"Std Deviation: {displacement_std:.6f} m")
            print(f"Range (Peak-to-Peak): {displacement_range:.6f} m")
            print(f"Max Displacement: {displacement_max:.6f} m, Min Displacement: {displacement_min:.6f} m")
            print("-" * 50)

            # Save the statistics for further analysis
            if participant not in statistics:
                statistics[participant] = []

            statistics[participant].append({
                'capture_name': capture_name,
                'channel': channel,
                'range_bin': range_bin,
                'mean_displacement': displacement_mean,
                'std_deviation': displacement_std,
                'range_displacement': displacement_range,
                'max_displacement': displacement_max,
                'min_displacement': displacement_min
            })

# Convert the statistics into a DataFrame for further analysis
stats_df = pd.DataFrame([
    {'participant': participant, **stat} for participant, stats_list in statistics.items() for stat in stats_list
])

print(stats_df)


    RADAR_capture  MOCAP_Start_Time  MOCAP_End_Time  RADAR_Start_Frame  \
0  01_MNTRL_RR_V1              7.33           30.57                194   
1  01_MNTRL_RR_V1              7.33           30.57                194   
2  01_MNTRL_RR_V1              7.33           30.57                194   
3  01_MNTRL_RR_V2              7.37           30.40                191   
4  01_MNTRL_RR_V2              7.37           30.40                191   

   RADAR_End_Frame  Seconds_per_Frame  tx  is_final_tx  t_foot_up  t_stable  \
0              831            0.03645   1        False      10.22     11.19   
1              831            0.03645   2        False      18.25     19.05   
2              831            0.03645   3         True      26.24     27.03   
3              822            0.03646   1        False      10.48     11.05   
4              822            0.03646   2        False      18.41     19.07   

   t_break  t_foot_down  frame_foot_up  frame_stable  frame_break  frame_end  
0

AttributeError: 'FMCWRADARDataCapture' object has no attribute 'unwrapped_microdoppler_phase_displacement'

In [ ]:
print(stats_df)

In [ ]:
# Example: Compare average standard deviation of displacement for each participant
avg_std_per_participant = stats_df.groupby('participant')['std_deviation'].mean()
print(avg_std_per_participant)

# Example: Compare the peak-to-peak range for each participant
avg_range_per_participant = stats_df.groupby('participant')['range_displacement'].mean()
print(avg_range_per_participant)


plots of phase analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualize displacement data for each participant
plt.figure(figsize=(12, 6))
sns.boxplot(x='participant', y='mean_displacement', data=stats_df)
plt.title('Boxplot of Mean Displacement for Each Participant')
plt.xlabel('Participant')
plt.ylabel('Mean Displacement [m]')
plt.show()


In [ ]:
from scipy.stats import shapiro

# Test normality for each participant
for participant in stats_df['participant'].unique():
    displacement_data = stats_df[stats_df['participant'] == participant]['mean_displacement']
    stat, p_value = shapiro(displacement_data)
    print(f"Participant {participant} - Shapiro-Wilk Test p-value: {p_value:.5f}")


In [ ]:
from scipy.stats import f_oneway

# Extract the displacement data for each participant
participant_groups = [stats_df[stats_df['participant'] == participant]['mean_displacement']
                      for participant in stats_df['participant'].unique()]

# Perform one-way ANOVA
stat, p_value = f_oneway(*participant_groups)
print(f"One-Way ANOVA p-value: {p_value:.5f}")

if p_value < 0.05:
    print("There is a significant difference between participants.")
else:
    print("There is no significant difference between participants.")


In [ ]:
from scipy.stats import kruskal

# Extract the displacement data for each participant
participant_groups = [stats_df[stats_df['participant'] == participant]['mean_displacement']
                      for participant in stats_df['participant'].unique()]

# Perform Kruskal-Wallis test
stat, p_value = kruskal(*participant_groups)
print(f"Kruskal-Wallis Test p-value: {p_value:.5f}")

if p_value < 0.05:
    print("There is a significant difference between participants.")
else:
    print("There is no significant difference between participants.")


wobble analysis

In [ ]:
import numpy as np
import pandas as pd

# Path to the CSV file containing time labels
time_labels_df_path = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/MOCAP_FP_RADAR_FU_Stable_Break_FD_TIME_FRAMES_v3.csv'
time_labels_df = pd.read_csv(time_labels_df_path)

output_folder_path = "/Volumes/FourTBLaCie/Yoga_Study_RADAR_AoA_Stability"
problem_list = []
wobble_stats = {}  # To store wobble stats per participant

for index, row in time_labels_df.iterrows():
    capture_name = row['RADAR_capture']

    participant = capture_name[:2]
        # Handle special cases to skip certain captures or transitions
    if capture_name == '22_MNTRL_RR_V1' and str(row['tx']) == "3":
        continue
    elif capture_name == '24_MNTRR_RR_V1' and str(row['tx']) == "2":
        continue
    elif capture_name == '12_MNTRR_RR_V1':
        continue   

    radar_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_RADAR_Data_Renamed/{participant}/{capture_name}.h5"

    try:
        # Initialize the FMCWRADARDataCapture object
        RADAR_object = FMCWRADARDataCapture(file_path=radar_file_path)
    except Exception as e:
        problem_list.append((capture_name, row['tx'], e))
        continue

    # Load the data
    dataCubes = RADAR_object.load_and_save()

    # Select frames between frame_stable and frame_end
    frame_stable = int(row['frame_stable'])
    if row['is_final_tx']:
        frame_end = int(row['frame_end'])
    else:
        frame_end = int(row['frame_break'])

    dataCubes_stable = dataCubes[:, frame_stable:frame_end, :, :]

    range_bins_of_interest = [7]

    # Compute the unwrapped phases or displacement
    displacements = RADAR_object.unwrapped_microdoppler_phase_displacement(
        capture_name=capture_name,
        tx=row['tx'],
        dataCubes=dataCubes_stable,
        range_bins=range_bins_of_interest,
        plot=False
    )

    # Compute wobble metrics for each channel and range bin
    for channel in displacements:
        for range_bin in displacements[channel]:
            t, displacement = displacements[channel][range_bin]

            # Compute wobble metrics
            displacement_std = np.std(displacement)  # Standard Deviation
            displacement_variance = np.var(displacement)  # Variance
            displacement_range = np.ptp(displacement)  # Peak-to-Peak range
            displacement_rms = np.sqrt(np.mean(displacement**2))  # RMS of Displacement

            # Print the wobble stats for the current channel, range bin, and participant
            print(f"Participant: {participant}, Capture: {capture_name}, Channel: {channel}, Range Bin: {range_bin}")
            print(f"Std Deviation: {displacement_std:.6f} m")
            print(f"Variance: {displacement_variance:.6f} m^2")
            print(f"Range (Peak-to-Peak): {displacement_range:.6f} m")
            print(f"RMS Displacement: {displacement_rms:.6f} m")
            print("-" * 50)

            # Save the wobble statistics for further analysis
            if participant not in wobble_stats:
                wobble_stats[participant] = []

            wobble_stats[participant].append({
                'capture_name': capture_name,
                'channel': channel,
                'range_bin': range_bin,
                'std_deviation': displacement_std,
                'variance': displacement_variance,
                'range_displacement': displacement_range,
                'rms_displacement': displacement_rms
            })

# Convert the wobble statistics into a DataFrame for further analysis
wobble_stats_df = pd.DataFrame([
    {'participant': participant, **stat} for participant, stats_list in wobble_stats.items() for stat in stats_list
])

# Display the DataFrame
print(wobble_stats_df)


In [ ]:
wobble_stats_df= stats_df

In [ ]:
# Perform Kruskal-Wallis test on the standard deviation of displacement (wobble measure)
from scipy.stats import kruskal

# Extract the standard deviation data for each participant
participant_groups_std = [wobble_stats_df[wobble_stats_df['participant'] == participant]['std_deviation']
                          for participant in wobble_stats_df['participant'].unique()]

# Perform Kruskal-Wallis test
stat, p_value = kruskal(*participant_groups_std)
print(f"Kruskal-Wallis Test on Standard Deviation p-value: {p_value:.5f}")

if p_value < 0.05:
    print("There is a significant difference in wobble (standard deviation) between participants.")
else:
    print("There is no significant difference in wobble between participants.")


In [ ]:
wobble_stats_df.head

Ranking Wobble

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import kruskal
import scikit_posthocs as sp
import matplotlib.pyplot as plt

# Assuming 'wobble_stats_df' contains the wobble statistics (standard deviation) from the previous steps

# Compute the average standard deviation of displacement for each participant
avg_wobble_per_participant = wobble_stats_df.groupby('participant')['std_deviation'].mean()

# Rank participants from most to least wobble based on average standard deviation
ranked_wobble = avg_wobble_per_participant.sort_values(ascending=False)

# Print the ranking
print("Participants ranked from most to least wobble (based on standard deviation):")
print(ranked_wobble)

# Perform Kruskal-Wallis test to determine if there is a significant difference in wobble between participants
participant_groups_std = [wobble_stats_df[wobble_stats_df['participant'] == participant]['std_deviation']
                          for participant in wobble_stats_df['participant'].unique()]

stat, p_value = kruskal(*participant_groups_std)
print(f"\nKruskal-Wallis Test on Standard Deviation p-value: {p_value:.5f}")

# If Kruskal-Wallis test shows significance, perform pairwise post-hoc test (Dunn's test)
if p_value < 0.05:
    print("There is a significant difference in wobble between participants.")
    
    # Perform Dunn's test for pairwise comparison after Kruskal-Wallis
    posthoc = sp.posthoc_dunn([wobble_stats_df[wobble_stats_df['participant'] == participant]['std_deviation']
                               for participant in wobble_stats_df['participant'].unique()], 
                              p_adjust='bonferroni')
    
    print("\nPairwise Dunn's Test Results (p-values):")
    print(posthoc)
    
   # Visualize the pairwise comparisons
    # Visualize the pairwise comparisons without passing 'xlabel' or 'ylabel' arguments directly
    plt.figure(figsize=(10, 6))
    sp.sign_plot(posthoc, ax=None, cbar=True, fmt=".2f")
    plt.title("Dunn's Test Pairwise Comparisons for Wobble (Standard Deviation)")
    plt.show()

else:
    print("There is no statistically significant difference in wobble between participants.")


participants_ranked = list(ranked_wobble.index)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Assuming posthoc contains the pairwise Dunn's test results
# Convert the posthoc matrix to a NumPy array for better handling
posthoc_array = posthoc.to_numpy()

# Create a heatmap with seaborn using an inverted color map
plt.figure(figsize=(10, 6))
ax = sns.heatmap(posthoc_array, annot=True, cmap="Greens_r", cbar=True, fmt=".2f",
                 xticklabels=participants_ranked, yticklabels=participants_ranked)

# Set axis labels and title
plt.xticks(rotation=45)
plt.title("Dunn's Test Pairwise Comparisons for Wobble (Standard Deviation)")
plt.tight_layout()  # Ensure the labels and layout fit well
plt.show()


Box Plots with Jittered Data Points

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.boxplot(x='participant', y='std_deviation', data=wobble_stats_df, showfliers=False)
sns.stripplot(x='participant', y='std_deviation', data=wobble_stats_df, color='black', jitter=0.1, size=4)
plt.title("Wobble Comparison by Participant (Box Plot with Data Points)")
plt.ylabel("Wobble (Standard Deviation)")
plt.xlabel("Participant")
plt.xticks(rotation=45)
plt.show()


Violin Plots

In [ ]:
plt.figure(figsize=(12, 6))
sns.violinplot(x='participant', y='std_deviation', data=wobble_stats_df, inner="quartile")
plt.title("Wobble Comparison by Participant (Violin Plot)")
plt.ylabel("Wobble (Standard Deviation)")
plt.xlabel("Participant")
plt.xticks(rotation=45)
plt.show()


Density Plot

In [ ]:
plt.figure(figsize=(10, 6))
for participant in participants_ranked:
    sns.kdeplot(wobble_stats_df[wobble_stats_df['participant'] == participant]['std_deviation'], label=participant)
plt.title("Wobble Density by Participant (KDE Plot)")
plt.ylabel("Density")
plt.xlabel("Wobble (Standard Deviation)")
plt.legend(title="Participant")
plt.show()


Bar Plot

In [ ]:
plt.figure(figsize=(10, 6))
ranked_wobble.plot(kind='bar', color='skyblue')
plt.title("Participants Ranked by Wobble (Standard Deviation)")
plt.ylabel("Wobble (Standard Deviation)")
plt.xlabel("Participant")
plt.xticks(rotation=45)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Example data (replace with your wobble_stats_df)
wobble_stats_df = pd.DataFrame({
    'participant': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15'],
    'std_deviation': [0.002, 0.005, 0.003, 0.003, 0.005, 0.003, 0.003, 0.003, 0.003, 0.004, 0.003, 0.004, 0.002, 0.002, 0.002]
})

# Create the box plot
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='participant', y='std_deviation', data=wobble_stats_df, showfliers=False)
sns.stripplot(x='participant', y='std_deviation', data=wobble_stats_df, color='black', jitter=0.1, size=4)

# Function to draw the significance bar
def add_significance_bar(x1, x2, y, h, text):
    plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c='k')  # Draw the bar
    plt.text((x1+x2)*.5, y+h, text, ha='center', va='bottom', color='k')  # Add the text

# Example of adding significance bars (replace with actual p-values)
significance_pairs = [("02", "05"), ("05", "10"), ("02", "10"), ("01", "13")]

# Convert participant labels to positions (needed for adding significance bars)
participants = sorted(wobble_stats_df['participant'].unique())
positions = {participant: i for i, participant in enumerate(participants_ranked)}

# Add significance bars with manually specified heights and significance levels
add_significance_bar(positions["02"], positions["05"], 0.0055, 0.0002, "**")
add_significance_bar(positions["05"], positions["10"], 0.006, 0.0002, "***")
add_significance_bar(positions["02"], positions["10"], 0.0065, 0.0002, "***")
add_significance_bar(positions["01"], positions["13"], 0.0045, 0.0002, "*")

# Add titles and labels
plt.title("Wobble Comparison by Participant (Box Plot with Significance Bars)")
plt.ylabel("Wobble (Standard Deviation)")
plt.xlabel("Participant")
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()


Unwrapped Phase by Bin Spectrogram

In [ ]:

# Now proceed with the code
# Path to the CSV file containing time labels
time_labels_df_path = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/MOCAP_FP_RADAR_FU_Stable_Break_FD_TIME_FRAMES_v3.csv'
time_labels_df = pd.read_csv(time_labels_df_path)

print(time_labels_df.head())

output_folder_path = "/Volumes/FourTBLaCie/Yoga_Study_RADAR_AoA_Stability"

problem_list = []

for index, row in time_labels_df.iterrows():
    print(row)

    participant = row['RADAR_capture'][:2]
    if participant != "04":
        continue

    radar_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_RADAR_Data_Renamed/{participant}/{row['RADAR_capture']}.h5"

    try:
        # Initialize the FMCWRADARDataCapture object
        RADAR_object = FMCWRADARDataCapture(file_path=radar_file_path)
    except Exception as e:
        problem_list.append((row['RADAR_capture'], row['tx'], e))
        continue

    # Load the data
    dataCubes = RADAR_object.load_and_save()
    
    print(dataCubes.shape)

    # Select frames between frame_stable and frame_end
    frame_stable = int(row['frame_stable'])
    if row['is_final_tx']:
        frame_end = int(row['frame_end'])
    else:
        frame_end = int(row['frame_break'])

    dataCubes_stable = dataCubes[:, frame_stable:frame_end, :, :]


    # Define the range bins of interest
    range_bins_of_interest = [6, 7, 8]  # Adjust based on your target's range

    # Compute the unwrapped phases
    unwrapped_phases = RADAR_object.unwrapped_microdoppler_phase_spectrogram(
        dataCubes=dataCubes,
        range_bins=range_bins_of_interest,
        nfft=1024,  # Adjust as needed
        plot=True  # Set to True to visualize the unwrapped phases
    )

    # Access the unwrapped phase data for further analysis
    channel = 0
    range_bin = 6
    frequencies, times, unwrapped_phase_data = unwrapped_phases[channel][range_bin]


Micro-doppler Spectrogram by Bin

In [ ]:

# Now proceed with the code
# Path to the CSV file containing time labels
time_labels_df_path = '/Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/csvs/MOCAP_FP_RADAR_FU_Stable_Break_FD_TIME_FRAMES_v3.csv'
time_labels_df = pd.read_csv(time_labels_df_path)

print(time_labels_df.head())

output_folder_path = "/Volumes/FourTBLaCie/Yoga_Study_RADAR_AoA_Stability"

problem_list = []

for index, row in time_labels_df.iterrows():
    
    print(row)
    
    # Uncomment the following line if you want to process a specific capture
    # if row['RADAR_capture'] != '12_MNTRL_RR_V1':
    #     continue
    
    participant = row['RADAR_capture'][:2]
    if participant != "04":
        continue
    
    radar_file_path = f"/Volumes/FourTBLaCie/Yoga_Study_RADAR_Data_Renamed/{participant}/{row['RADAR_capture']}.h5"
    
    try:
        # Initialize the FMCWRADARDataCapture object
        RADAR_object = FMCWRADARDataCapture(file_path=radar_file_path)
    except Exception as e:
        problem_list.append((row['RADAR_capture'], row['tx'], e))
        continue
    
    # Load the data
    dataCubes = RADAR_object.load_and_save()

    # Select frames between frame_stable and frame_end
    frame_stable = int(row['frame_stable'])
    if row['is_final_tx']:
        frame_end = int(row['frame_end'])
    else:
        frame_end = int(row['frame_break'])
    
    dataCubes_stable = dataCubes[:, frame_stable:frame_end, :, :]

    # Now, compute micro-Doppler signatures
    # Define range bins of interest
    range_bins_of_interest = [6, 7, 8]  # Adjust these values based on your application

    # Compute micro-Doppler signatures
    microdoppler_signatures = RADAR_object.compute_microdoppler_signature(
        dataCubes=dataCubes_stable,
        range_bins=range_bins_of_interest,
        nfft=256,
        plot=True  # Set to True to plot the spectrograms
    )

    # # If needed, save the micro-Doppler signatures
    # # For example, save the spectrogram data for each channel and range bin
    # for channel in microdoppler_signatures:
    #     for range_bin in microdoppler_signatures[channel]:
    #         f, t, Sxx = microdoppler_signatures[channel][range_bin]
    #         # Save the spectrogram data
    #         # Create an output filename
    #         output_filename = os.path.join(
    #             output_folder_path,
    #             participant,
    #             f"{row['RADAR_capture']}_channel_{channel}_rangebin_{range_bin}_microdoppler.npz"
    #         )
    #         # Ensure the output directory exists
    #         os.makedirs(os.path.dirname(output_filename), exist_ok=True)
    #         # Save the data
    #         np.savez(output_filename, frequencies=f, times=t, spectrogram=Sxx)
    #         print(f"Saved micro-Doppler spectrogram to {output_filename}")